<a href="https://colab.research.google.com/github/ArtemioPadilla/MachineLearningAndCOVID/blob/main/src/NNClassifiers/NeuralNetworkClassifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch

# Data download

Epidemiologic surveillance system of Mexico City:

https://datos.cdmx.gob.mx/dataset/base-covid-sinave

In [3]:
#! wget -c "https://archivo.datos.cdmx.gob.mx/sisver/sisver_public.csv"

# Import Libraries

In [4]:
import pandas as pd
import numpy as np
import random

In [5]:

import matplotlib.pyplot as plt


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve, auc
from sklearn import metrics

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print (f"Hardware disponible: {device}\n")

Hardware disponible: cuda



# Import Data

In [7]:
df = pd.read_csv("sisver_public.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (76,78,79,85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df.shape

(3793152, 92)

# Preprocessing

Lets create a subset only with positive or negative covid cases:

In [9]:
df = df[df['resdefin'].isin(['NEGATIVO','SARS-CoV-2'])] 

Lets now select the most interesting fields

In [10]:
columns = ['fechreg', 'sexo', 'tipacien', 'fecdef', 'intubado', 'digcline', 'edad', 'fiebre', 'tos', 'odinogia', 'disnea', 'irritabi', 'diarrea', 
           'dotoraci','calofrios', 'cefalea','mialgias', 'artral', 'ataedoge', 	'rinorrea', 	'polipnea', 	'vomito', 	'dolabdo', 	'conjun', 	'cianosis',
           'inisubis', 	'diabetes', 	'epoc', 	'asma', 	'inmusupr', 	'hiperten', 	'vih_sida', 	'otracon', 	'enfcardi', 	'obesidad', 	'insrencr',
           'tabaquis', 'resdefin']

In [11]:
labels_translation = {"sexo":"Sex", "tipacien":"TypeOfPatient","fecdef":"DefunctionDate",
                      "digcline":"Pneumonia", "edad":"Age", "fiebre":"Fever","tos":"Cough",
                      "odinogia":"Odynophagia", "disnea":"ShortnessOfBreath", "irritabi":"Irritability",
                      "diarrea":"Diarrhea", 'dotoraci':"ThoracicPain",'calofrios':"Chills", 
                      'cefalea':"Headache",'mialgias':"Myalgia", 'artral':"Arthralgia", 
                      'ataedoge':"PoorGeneralCondition", 'rinorrea':"Rhinorrhea", 	
                      'polipnea':"Polypnea", 	'vomito':"Vomit", 	'dolabdo':"AbdominalPain",
                      'conjun':"Conjunctivitis", 	'cianosis':"Cyanosis", 'inisubis':"SuddenSyndromStart",
                      'diabetes':"Diabetes", 	'epoc':"COPD", 	'asma':"Asthma", 	'inmusupr':"Immunosuppression",
                      'hiperten':"Hypertension", 	'vih_sida':"HIV/AIDS", 	'otracon':"OtherCommorbidity",
                      'enfcardi':"CardiovascularDisease", 	'obesidad':"Obesity", 'insrencr':"ChronicKidneyDisease",
                      'tabaquis':"Smoking", 'resdefin':"COVID-19"}

Lets create a subset with this fields:

In [12]:
dfmini = df.loc[:,columns]
print(dfmini.shape)
dfmini.head()

(3692986, 38)


,fechreg,sexo,tipacien,fecdef,intubado,digcline,edad,fiebre,tos,odinogia,...,asma,inmusupr,hiperten,vih_sida,otracon,enfcardi,obesidad,insrencr,tabaquis,resdefin
0,2021-01-15T00:00:00Z,FEMENINO,AMBULATORIO,NaN,NaN,NO,34,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NEGATIVO
1,2021-10-08T00:00:00Z,FEMENINO,AMBULATORIO,NaN,NaN,NO,44,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NEGATIVO
2,2021-07-08T00:00:00Z,MASCULINO,AMBULATORIO,NaN,NaN,NO,32,NO,SI,SI,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NEGATIVO
3,2022-01-28T00:00:00Z,FEMENINO,AMBULATORIO,NaN,NaN,NO,39,NO,SI,SI,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NEGATIVO
4,2021-08-15T00:00:00Z,FEMENINO,AMBULATORIO,NaN,NaN,NO,38,NO,SI,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NEGATIVO


Lets check for empty values:

In [13]:
nulls = (dfmini.isnull().sum()/len(dfmini)).sort_values(ascending=False)[:5]
print(nulls)

fecdef      0.993265
intubado    0.975202
digcline    0.000357
fechreg     0.000000
inmusupr    0.000000
dtype: float64


Lets fill the NaNs with default values

    fecdef (defunction date) -> if empty hasn't die

    intubado (intubated)  -> if empty wasn't intubated
    
    digcline (clinic diagnoses of Pneumonia)  -> if empty was not diagnosed


In [14]:
dfmini["fecdef"].fillna("NO", inplace=True)
dfmini["intubado"].fillna("NO", inplace=True)
dfmini["digcline"].fillna("NO", inplace=True)
# Lets change defunction dates to a binary indicator of whether the patient died or not
dfmini["fecdef"] = dfmini["fecdef"].map(lambda x: "SI" if x != "NO" else x)

Lets check again for null values

In [15]:
(dfmini.isnull().sum()/len(dfmini)).sort_values(ascending=False)[:5]

fechreg    0.0
asma       0.0
vomito     0.0
dolabdo    0.0
conjun     0.0
dtype: float64

In [16]:
dfmini.diabetes.unique(), dfmini.shape

(array(['NO', 'SI', 'SE IGNORA'], dtype=object), (3692986, 38))

Lets remove unknown data:

In [17]:
to_check = ['fiebre', 'tos', 'odinogia', 'disnea', 'irritabi', 'diarrea', 'dotoraci',
            'calofrios', 'cefalea','mialgias', 'artral', 'ataedoge', 	'rinorrea', 	
            'polipnea', 	'vomito', 	'dolabdo', 	'conjun', 	'cianosis','inisubis', 	
            'diabetes', 	'epoc', 	'asma', 	'inmusupr', 	'hiperten', 	'vih_sida', 	
            'otracon', 	'enfcardi', 	'obesidad', 	'insrencr','tabaquis']

for col in to_check:
  dfmini = dfmini[dfmini[col] != 'SE IGNORA']

dfmini.shape

(3659727, 38)

In [18]:
dfmini.diabetes.unique(), dfmini.shape

(array(['NO', 'SI'], dtype=object), (3659727, 38))

Lets change the boolean word of "No" and "Sí" for 0s and 1s.

In [19]:
cat = {}
for col in dfmini:
  if col in ["fechreg", "edad"]: continue 
  map = {x:i for i,x in enumerate(dfmini[col].unique())}
  cat[col] = map
  dfmini[col] = dfmini[col].map(map)

dfmini.head()

,fechreg,sexo,tipacien,fecdef,intubado,digcline,edad,fiebre,tos,odinogia,...,asma,inmusupr,hiperten,vih_sida,otracon,enfcardi,obesidad,insrencr,tabaquis,resdefin
0,2021-01-15T00:00:00Z,0,0,0,0,0,34,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-10-08T00:00:00Z,0,0,0,0,0,44,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-07-08T00:00:00Z,1,0,0,0,0,32,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2022-01-28T00:00:00Z,0,0,0,0,0,39,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2021-08-15T00:00:00Z,0,0,0,0,0,38,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
dfmini = dfmini.sample(2000000, random_state=42) # two million registers

Lets make a dataset for symptoms and covid-19 diagnoses:

In [21]:
# Lets create a dataframe to try to detect covid given the symptoms
col_sin = ['sexo', 'edad', 'fiebre', 'tos', 'odinogia', 'disnea', 'irritabi', 'diarrea', 
           'dotoraci','calofrios', 'cefalea', 'mialgias', 'artral', 'ataedoge', 'rinorrea', 'polipnea',	'vomito', 'dolabdo', 'conjun', 'cianosis',
           'inisubis', 'resdefin'] #'tipacien', 'intubado', 'digcline', 

df_sin = dfmini.loc[:,col_sin]
df_sin.head(), df_sin.shape

(         sexo  edad  fiebre  tos  odinogia  disnea  irritabi  diarrea  \
 1173599     1    49       1    0         0       0         0        0   
 2712989     0    58       0    0         0       0         0        0   
 141825      0    51       0    1         0       0         0        0   
 1003142     1    32       0    0         0       0         0        0   
 3598245     1    29       0    0         0       0         0        1   
 
          dotoraci  calofrios  ...  artral  ataedoge  rinorrea  polipnea  \
 1173599         0          0  ...       1         0         0         0   
 2712989         0          0  ...       0         0         0         0   
 141825          1          1  ...       0         0         0         0   
 1003142         0          0  ...       0         0         0         0   
 3598245         0          0  ...       0         0         0         0   
 
          vomito  dolabdo  conjun  cianosis  inisubis  resdefin  
 1173599       0        0     

Lets make a dataframe for the comorbidities and defuntion outcome given a pacient is positive to covid-19.

In [22]:
# Lets create a dataframe for deaths given the comorbidities
col_com  = ['sexo', 'edad', 'diabetes',	'epoc', 'asma',	'inmusupr', 
            'hiperten', 'vih_sida', 'otracon', 'enfcardi', 'obesidad', 'insrencr',
            'tabaquis', 'resdefin', 'fecdef']

df_com = dfmini.loc[:,col_com]
df_com = df_com[df_com['resdefin']==1]
df_com.drop('resdefin', axis=1, inplace=True)
df_com.head(), df_com.shape

(         sexo  edad  diabetes  epoc  asma  inmusupr  hiperten  vih_sida  \
 1173599     1    49         0     0     0         0         0         0   
 1985058     0    11         0     0     0         0         0         0   
 3231131     1    26         0     0     0         0         0         0   
 476817      1    29         0     0     0         0         0         0   
 1104026     1    23         0     0     0         0         0         0   
 
          otracon  enfcardi  obesidad  insrencr  tabaquis  fecdef  
 1173599        0         0         0         0         0       0  
 1985058        0         0         0         0         0       0  
 3231131        0         0         0         0         0       0  
 476817         0         0         0         0         1       0  
 1104026        0         0         0         0         0       0  ,
 (431112, 14))

Lets now make a dataframe to try to predict if a pacient will be hospitalized given his symptoms and comorbidities:

Tratamos de predecir si el paciente quedara hospitalizado a traves de sus sintomas y comorbilidades, observe que el hecho de que el paciente este intubado implica que esta hospitalizado, asi que creamos un nuevo dataframe borrando esta columnas, asi como las columnas, tippacien,fecdef,resdefin y tomamos la columna $y$ como la columna que nos dice si el paciente fue hospitalizado o no

In [23]:
# Lets create a dataframe for hospitalizations given symptoms and comorbidities
symptoms_and_com = ['sexo', 'digcline', 'edad', 'fiebre', 'tos', 'odinogia', 'disnea', 'irritabi', 'diarrea', 
           'dotoraci','calofrios', 'cefalea','mialgias', 'artral', 'ataedoge', 	'rinorrea', 	'polipnea', 	'vomito', 	'dolabdo', 	'conjun', 	'cianosis',
           'inisubis', 	'diabetes', 	'epoc', 	'asma', 	'inmusupr', 	'hiperten', 	'vih_sida', 	'otracon', 	'enfcardi', 	'obesidad', 	'insrencr',
           'tabaquis', 'resdefin', 'tipacien']

In [24]:
df_hosp = dfmini.loc[:,symptoms_and_com] # Symptoms and comorbidities
df_hosp = df_hosp[df_hosp.resdefin == 1]  # Given the patient has covid
df_hosp.drop('resdefin', axis=1, inplace=True)
df_hosp

,sexo,digcline,edad,fiebre,tos,odinogia,disnea,irritabi,diarrea,dotoraci,...,asma,inmusupr,hiperten,vih_sida,otracon,enfcardi,obesidad,insrencr,tabaquis,tipacien
1173599,1,0,49,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1985058,0,0,11,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3231131,1,0,26,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476817,1,0,29,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1104026,1,0,23,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068960,0,1,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1203167,0,0,44,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
330,1,0,46,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
169046,1,0,24,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**DATAFRAMES SO FAR**
- df_sin: dataframe with only the symptoms to try to detect covid
- df_com: dataframe with only the comorbidities to try to predict death
- df_hosp: dataframe with the symptoms and comorbidities to try to predict hospitalization

**The symptoms are:**
fiebre, tos, odinogia, disnea, irritabi, diarrea, dotoraci, calofrios, cefalea, mialgias, artral, ataedoge, rinorrea, polipnea, vomito, dolabdo, conjun, cianosis, inisubis

**The comorbidities are:**
diabetes, epoc, asma, inmusupr, hiperten, vih_sida, otracon, enfcardi, obesidad, insrencr, tabaquis

# Neural Networks Classifier

In [25]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import torch.optim as optim

## COVID Diagnosis

In [26]:

#y = df.loc[:, "resdefin"].values
#X = df.drop("resdefin", axis=1).values


In [27]:
def to_numpy_X_y(df , y_col):
  Y = df.loc[:, y_col].values
  X = df.drop(y_col, axis=1).values

  print (f"Longitud X: {len(X)}")
  print (f"Longitud Y: {len(Y)}")
  return X, Y

In [28]:
# dividir el conjunto de entrenamiento en Train and Test

def ttv_split(X, Y, p_test, p_val):

  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=p_test+p_val, stratify=Y, random_state=0)
  X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=p_val/(p_test+p_val), stratify=Y_test, random_state=0)

  print(f"X_train.shape, Y_train.shape: {X_train.shape}, {Y_train.shape}")
  print(f"X_test.shape, Y_test.shape: {X_test.shape}, {Y_test.shape}")
  print(f"X_val.shape, Y_val.shape: {X_test.shape}, {Y_test.shape}")
  return X_train, X_test, X_val, Y_train, Y_test, Y_val

In [29]:
def to_torch(X_train, X_test, X_val, Y_train, Y_test, Y_val):
  # crear los tensores del conjunto de datos
  train = TensorDataset(torch.tensor(X_train)[:,None, :], torch.tensor(Y_train))
  test = TensorDataset(torch.tensor(X_test)[:,None, :], torch.tensor(Y_test))
  val = TensorDataset(torch.tensor(X_val)[:,None, :], torch.tensor(Y_val))

  # crear los cargadores del conjunto de datos
  trainloader = DataLoader(train, batch_size=100, shuffle=True)
  testloader = DataLoader(test, batch_size=100, shuffle=True)
  valloader = DataLoader(val, batch_size=100, shuffle=True)
  return trainloader, testloader, valloader

## Modelo

In [30]:
class NNcovid_diagnosis(nn.Module):
  
  def __init__(self, input_dim, hidden_dim, labels_dim):
    super(NNcovid_diagnosis, self).__init__()

    # input_dim = máxima longitud de tweets  

    # definimos las capas de convolución (CNN)
    self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
    self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
    

    self.fc1 = nn.Linear(32*21, hidden_dim)
    # definir una función no líneal de activación 1
    self.activation1 = nn.Sigmoid()
    
    self.fc2 = nn.Linear(hidden_dim, hidden_dim)
    # definir una función no líneal de activación 2
    self.activation2 = nn.Sigmoid()

    # Linear function 3 (readout): 500 --> 3
    self.fc3 = nn.Linear(hidden_dim, labels_dim)
    # Dropout with p=0.2
    self.dropout = nn.Dropout(p=0.2)

  def forward(self, x):
    # propagación hacia adelante

    # x: [batchsize, 1, input_dim = 21]

    out = self.conv1(x)        # [batchsize, 16, 21]
    #out = F.max_pool1d(out, kernel_size=2)        # [batchsize, 16, 21]
    out = F.relu(out)       # [batchsize, 16, 21]

    out = self.conv2(out)        # [batchsize, 32, 21]
    #out = F.max_pool1d(out, kernel_size=2)        # [batchsize, 32, 21]
    out = F.relu(out)       # [batchsize, 32, 21]

    # aplanar (flatenning)
    out = out.view(out.size(0), -1) # [batchsize, 32*21 = 864]

    out = self.fc1(out)

    # Non-linearity 1
    out = self.activation1(out)

    # Linear function 2
    out = self.fc2(out)

    # Non-linearity 2
    out = self.activation2(out)

    # Linear function 3 (readout)
    out = self.fc3(out)
    
    out = self.dropout(out)

    out = F.log_softmax(out, dim=1)

    return out

In [31]:
class NNcovid_hosp(nn.Module):
  
  def __init__(self, input_dim, hidden_dim, labels_dim):
    super(NNcovid_hosp, self).__init__()

    # input_dim = máxima longitud de tweets  

    # definimos las capas de convolución (CNN)
    self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
    self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
    

    self.fc1 = nn.Linear(32*21, hidden_dim)
    # definir una función no líneal de activación 1
    self.activation1 = nn.Sigmoid()
    
    self.fc2 = nn.Linear(hidden_dim, hidden_dim)
    # definir una función no líneal de activación 2
    self.activation2 = nn.Sigmoid()

    # Linear function 3 (readout): 500 --> 3
    self.fc3 = nn.Linear(hidden_dim, labels_dim)
    # Dropout with p=0.2
    self.dropout = nn.Dropout(p=0.2)

  def forward(self, x):
    # propagación hacia adelante

    # x: [batchsize, 1, input_dim = 21]

    out = self.conv1(x)        # [batchsize, 16, 21]
    #out = F.max_pool1d(out, kernel_size=2)        # [batchsize, 16, 21]
    out = F.relu(out)       # [batchsize, 16, 21]

    out = self.conv2(out)        # [batchsize, 32, 21]
    #out = F.max_pool1d(out, kernel_size=2)        # [batchsize, 32, 21]
    out = F.relu(out)       # [batchsize, 32, 21]

    # aplanar (flatenning)
    out = out.view(out.size(0), -1) # [batchsize, 32*21 = 864]

    out = self.fc1(out)

    # Non-linearity 1
    out = self.activation1(out)

    # Linear function 2
    out = self.fc2(out)

    # Non-linearity 2
    out = self.activation2(out)

    # Linear function 3 (readout)
    out = self.fc3(out)
    
    out = self.dropout(out)

    out = F.log_softmax(out, dim=1)

    return out

In [32]:
# Instanciar el modelo
def instanciate_model(Model, input_dim, hidden_dim = 500, labels_dim=2):

  model = Model(input_dim, hidden_dim, labels_dim)
  model.to(device)
  return model

In [33]:
def train(train_loader, val_loader, model, lr=0.01, EPOCHS = 10):
  history={
    'train_loss': [],
    'valid_loss': [],
    'train_acc': [],
    'valid_acc': []
  }

  print(f"EPOCHS: {EPOCHS}")

  # definir la función de error
  loss_function = nn.CrossEntropyLoss()


  optimizer = optim.Adam(model.parameters(), lr=lr)
  
  for epoch in range(EPOCHS):
    train_loss, train_correct = 0.0, 0

    # indicamos que se calculará el gradiente (Entrenamiento) 
    model.train()

    # recorremos el conjunto de entrenamiento en los batches definidos
    for X, y in train_loader:
      # definir el hardware disponible a las imágenes y a las clases
      X,y = X.to(device), y.to(device)
      X = X.float()
      
      optimizer.zero_grad()

      output = model(X) 
      #target = y.squeeze(1)#.clone().detach().long()
      
      #print(y.shape, target.shape)
      loss = loss_function(output,y)
      loss.backward()
      optimizer.step()
      train_loss += loss.item() * X.size(0)
      scores, predictions = torch.max(output.data, 1)
      train_correct += (predictions == y).sum().item()    
      
    valid_loss, val_correct = 0.0, 0

    # indicamos que no se calculará el gradiente (Validación) 
    model.eval()
    
    for X, y in val_loader:
      X, y = X.to(device), y.to(device)
      X = X.float()
      output = model(X)
      #target = y.squeeze(1)#.clone().detach().long()
      loss = loss_function(output, y)
      valid_loss+=loss.item()*X.size(0)
      scores, predictions = torch.max(output.data,1)
      val_correct+=(predictions == y).sum().item()

    # calcular los promedios tanto de la pérdida como de la excatitud en ambos datasets (train, valid)
    train_loss = train_loss / len(train_loader.sampler)
    train_acc = train_correct / len(train_loader.sampler)*100
    valid_loss = valid_loss / len(val_loader.sampler)
    valid_acc = val_correct / len(val_loader.sampler) * 100

    if epoch % 1 == 0 or epoch + 1 == EPOCHS:
      print(f"Epoch:{epoch+1}/{EPOCHS}\tTrain Loss:{train_loss:.4f} Validation Loss:{valid_loss:.4f}\tTrain Accuracy:{train_acc:.2f}% Validation Accuracy:{valid_acc:.2f}%")

    history['train_loss'].append(train_loss)
    history['valid_loss'].append(valid_loss)
    history['train_acc'].append(train_acc)
    history['valid_acc'].append(valid_acc)
  return history

In [34]:
def plot_scores(history):
  plt.figure(figsize=(30, 10))
  plt.subplot(1, 2, 1)
  plt.plot(history['train_loss'])
  plt.plot(history['valid_loss'])
  plt.title('Model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train_loss', 'val_loss'], loc='best')
  # plt.show()

  plt.subplot(1, 2, 2)
  plt.plot(history['train_acc'])
  plt.plot(history['valid_acc'])
  plt.title('Model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train_acc', 'val_acc'], loc='best')
  plt.show()

In [35]:
def evaluar(model, test_loader):
  correct = 0
  total = 0
  test_loss = 0

  # definir la función de pérdida: Entropía Cruzada
  loss_function = nn.CrossEntropyLoss()

  with torch.no_grad():
    for X, y in test_loader:
      X, y = X.to(device), y.to(device)
      X = X.float()
      outputs = model(X)

      #target = y.squeeze(1).clone().detach().long()
      loss = loss_function(outputs,y)
      test_loss += loss.item() * X.size(0)
      
      _, predicted = torch.max(outputs.data, 1)
      total += y.size(0)
      correct += (predicted == y).sum().item()

  test_loss = test_loss / len(test_loader.sampler)

  print(f'Test Accuracy : {100 * correct / total:.2f}%')
  print(f'Test Loss: {test_loss:.4f}')

# Classifier Pipeline

In [36]:
def pipeline(Model, df, y_col, lr = 0.01, EPOCHS = 10):
  torch.manual_seed(0)
  random.seed(0)
  np.random.seed(0)

  X, Y = to_numpy_X_y(df, y_col)
  input_dim = X.shape[1]

  X_train, X_test, X_val, Y_train, Y_test, Y_val = ttv_split(X, Y, 0.2, 0.2)
  
  train_loader, test_loader, val_loader = to_torch(X_train, X_test, X_val, Y_train, Y_test, Y_val)

  model = instanciate_model(Model, input_dim = input_dim, hidden_dim = 500)

  history = train(train_loader, val_loader, model,  lr, EPOCHS)

  plot_scores(history)

  evaluar(model, test_loader)

  return model, X_train, X_test, X_val, Y_train, Y_test, Y_val

### Helper functions

In [37]:
def predict_model_diagnosis(model_diagnosis, X = np.zeros((1, 21)), device = 'cpu'):
  _, pred = torch.max(model_diagnosis(torch.tensor(X)[:,None, :].to(device).float()), 1)
  return pred.cpu().numpy()[0]
  
def prob_model_diagnosis(model_diagnosis, X = np.zeros((1, 21)), device = 'cpu'):
  log_probs = model_diagnosis(torch.tensor(X)[:,None, :].to(device).float())
  return np.exp(log_probs.cpu().detach().numpy())[0]

## COVID-19 diagnosis

In [ ]:
model, X_train, X_test, X_val, Y_train, Y_test, Y_val = pipeline(NNcovid_diagnosis, df_sin, "resdefin", lr = 0.0001, EPOCHS=20)

Longitud X: 2000000
Longitud Y: 2000000
X_train.shape, Y_train.shape: (1200000, 21), (1200000,)
X_test.shape, Y_test.shape: (400000, 21), (400000,)
X_val.shape, Y_val.shape: (400000, 21), (400000,)
EPOCHS: 20
Epoch:1/20	Train Loss:0.4770 Validation Loss:0.4528	Train Accuracy:78.85% Validation Accuracy:79.09%
Epoch:2/20	Train Loss:0.4731 Validation Loss:0.4515	Train Accuracy:78.94% Validation Accuracy:79.04%
Epoch:3/20	Train Loss:0.4719 Validation Loss:0.4487	Train Accuracy:79.01% Validation Accuracy:79.00%
Epoch:4/20	Train Loss:0.4714 Validation Loss:0.4495	Train Accuracy:79.04% Validation Accuracy:79.06%
Epoch:5/20	Train Loss:0.4708 Validation Loss:0.4479	Train Accuracy:79.05% Validation Accuracy:79.14%
Epoch:6/20	Train Loss:0.4704 Validation Loss:0.4484	Train Accuracy:79.08% Validation Accuracy:79.17%
Epoch:7/20	Train Loss:0.4702 Validation Loss:0.4486	Train Accuracy:79.11% Validation Accuracy:79.23%
Epoch:8/20	Train Loss:0.4698 Validation Loss:0.4476	Train Accuracy:79.11% Validation

In [ ]:
_, predicted = torch.max(model(torch.tensor(X_val)[:,None, :].to(device).float()), 1)

In [ ]:
print( classification_report(Y_val, predicted.cpu().numpy(), target_names=["COVID","NEGATIVE"]) )

In [ ]:
torch.save(model, "model_diagnosis.pth")

In [ ]:
m_diagnosis = torch.load("model_diagnosis.pth")

## Hospitalization

In [ ]:
df_hosp.tipacien

In [ ]:
model_hosp, X_train, X_test, X_val, Y_train, Y_test, Y_val = pipeline(df_hosp, "tipacien", lr = 0.0001, EPOCHS=20)

# Use cases

## Covid Diagnosis

In [ ]:
sexo = 1
edad = 70
fiebre = 1
tos = 1
odinogia = 1
disnea = 1
irritabi = 1
diarrea = 1
dotoraci = 1
calofrios = 1
cefalea = 1
mialgias = 1
artral = 1
ataedoge = 1
rinorrea = 1
polipnea = 1
vomito = 1
dolabdo = 1
conjun = 1
cianosis = 1
inisubis = 1

X = np.array([[sexo, edad, fiebre, tos, odinogia, disnea, irritabi,
  diarrea, dotoraci, calofrios, cefalea, mialgias, artral,
  ataedoge, rinorrea, polipnea, vomito, dolabdo, conjun,
  cianosis, inisubis]])

res = predict_model_diagnosis(model, X, 'cuda') 


probs = prob_model_diagnosis(model, X, 'cuda') 

if res == 1:
    print(f"El clasificador indica que eres POSITIVO a COVID-19 con probabilidad de {probs[1]:.2f}%")
elif res == 0:
    print(f"El clasificador indica que eres NEGATIVO a COVID-19 con probabilidad de {probs[0]:.2f}%")